In [31]:
import pandas as pd 
from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, CHAR, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import sqlite3


## 1- Extraction:
Retrieving data via the API using the requests library.

In [10]:
import requests
url = 'https://fakestoreapi.com/products'
r = requests.get(url)
data = r.json()

In [4]:
print(r.json())

[{'id': 1, 'title': 'Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops', 'price': 109.95, 'description': 'Your perfect pack for everyday use and walks in the forest. Stash your laptop (up to 15 inches) in the padded sleeve, your everyday', 'category': "men's clothing", 'image': 'https://fakestoreapi.com/img/81fPKd-2AYL._AC_SL1500_.jpg', 'rating': {'rate': 3.9, 'count': 120}}, {'id': 2, 'title': 'Mens Casual Premium Slim Fit T-Shirts ', 'price': 22.3, 'description': 'Slim-fitting style, contrast raglan long sleeve, three-button henley placket, light weight & soft fabric for breathable and comfortable wearing. And Solid stitched shirts with round neck made for durability and a great fit for casual fashion wear and diehard baseball fans. The Henley style round neckline includes a three-button placket.', 'category': "men's clothing", 'image': 'https://fakestoreapi.com/img/71-3HjGNDUL._AC_SY879._SX._UX._SY._UY_.jpg', 'rating': {'rate': 4.1, 'count': 259}}, {'id': 3, 'title': 'Mens Cotto

## 2- Transformation:
Converting the data into a DataFrame using Pandas and then performing some data cleaning.

In [21]:
import csv

headers = data[0].keys()

with open('products_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(data)

df = pd.read_csv('products_data.csv')

In [12]:
df.head()

,id,title,price,description,category,image,rating
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,"{'rate': 3.9, 'count': 120}"
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,"{'rate': 4.1, 'count': 259}"
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,"{'rate': 4.7, 'count': 500}"
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,"{'rate': 2.1, 'count': 430}"
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,"{'rate': 4.6, 'count': 400}"


In [22]:
df = df.rename(columns={'title': 'product_name'})

In [23]:
df = df.drop('image', axis=1)

In [17]:
df.head()

,id,product_name,price,description,category,rating
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,"{'rate': 3.9, 'count': 120}"
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,"{'rate': 4.1, 'count': 259}"
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,"{'rate': 4.7, 'count': 500}"
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,"{'rate': 2.1, 'count': 430}"
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,"{'rate': 4.6, 'count': 400}"


In [25]:
import ast
df['rating'] = df['rating'].apply(ast.literal_eval)

df['rating_rate'] = df['rating'].apply(lambda x: x['rate'] if isinstance(x, dict) else None)
df['rating_count'] = df['rating'].apply(lambda x: x['count'] if isinstance(x, dict) else None)


In [27]:
df.drop(columns=['rating'], inplace=True)

In [28]:
df.head()

,id,product_name,price,description,category,rating_rate,rating_count
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,3.9,120
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,4.1,259
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,4.7,500
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,2.1,430
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,4.6,400


## 3- Loading:
Using SQLAlchemy to Create a local SQLite database and insert the data after creating a "products" table.

In [ ]:
engine = create_engine('sqlite:///sales_data.db')
Base = declarative_base()


In [32]:
class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    title = Column(String)
    price = Column(Float)
    category = Column(String)
    description = Column(String)
    rating_rate = Column(Float)
    rating_count = Column(Integer)




In [33]:
Base.metadata.create_all(engine)


In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
for index, row in df.iterrows():
    product = Product(
        id=row['id'],
        title=row['product_name'],
        price=row['price'],
        category=row['category'],
        description=row['description'],
        rating_rate=row['rating_rate'],
        rating_count=row['rating_count']
    )
    session.add(product)

session.commit()

existing = session.query(Product).filter_by(id=row['id']).first()
if not existing:
    session.add(product)

## 4- Data Querying:
Using SQLAlchemy ORM to run SQL queries in Python to explore the data.

In [ ]:
products = session.query(Product).all()

print(products[0].__dict__) 


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000167B1325850>, 'id': 1, 'description': 'Your perfect pack for everyday use and walks in the forest. Stash your laptop (up to 15 inches) in the padded sleeve, your everyday', 'title': 'Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops', 'rating_count': 120, 'category': "men's clothing", 'price': 109.95, 'rating_rate': 3.9}


In [38]:
for prodcut in products:
    print(product.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000167B1326610>, 'id': 20, 'description': '95%Cotton,5%Spandex, Features: Casual, Short Sleeve, Letter Print,V-Neck,Fashion Tees, The fabric is soft and has some stretch., Occasion: Casual/Office/Beach/School/Home/Street. Season: Spring,Summer,Autumn,Winter.', 'title': 'DANVOUY Womens T Shirt Casual Cotton Short', 'rating_count': 145, 'category': "women's clothing", 'price': 12.99, 'rating_rate': 3.6}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000167B1326610>, 'id': 20, 'description': '95%Cotton,5%Spandex, Features: Casual, Short Sleeve, Letter Print,V-Neck,Fashion Tees, The fabric is soft and has some stretch., Occasion: Casual/Office/Beach/School/Home/Street. Season: Spring,Summer,Autumn,Winter.', 'title': 'DANVOUY Womens T Shirt Casual Cotton Short', 'rating_count': 145, 'category': "women's clothing", 'price': 12.99, 'rating_rate': 3.6}
{'_sa_instance_state': <sqlalchemy.orm.state

Count of products


In [47]:
product_count = session.query(Product).count()

print(f"Total products: {product_count}")


Total products: 20


Expansive products

In [ ]:
expensive_products = session.query(Product).filter(Product.price > 50).all()

for product in expensive_products:
    print(product.__dict__)



{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000167B1325850>, 'id': 1, 'description': 'Your perfect pack for everyday use and walks in the forest. Stash your laptop (up to 15 inches) in the padded sleeve, your everyday', 'title': 'Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops', 'rating_count': 120, 'category': "men's clothing", 'price': 109.95, 'rating_rate': 3.9}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000167B1324350>, 'id': 3, 'description': 'great outerwear jackets for Spring/Autumn/Winter, suitable for many occasions, such as working, hiking, camping, mountain/rock climbing, cycling, traveling or other outdoors. Good gift choice for you or your family member. A warm hearted love to Father, husband or son in this thanksgiving or Christmas Day.', 'title': 'Mens Cotton Jacket', 'rating_count': 500, 'category': "men's clothing", 'price': 55.99, 'rating_rate': 4.7}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceS

Exploring electronics products

In [45]:
electronics_products = session.query(Product).filter(Product.category == 'electronics').all()

for product in electronics_products:
    print(product.__dict__)



{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000167B1325690>, 'id': 9, 'description': 'USB 3.0 and USB 2.0 Compatibility Fast data transfers Improve PC Performance High Capacity; Compatibility Formatted NTFS for Windows 10, Windows 8.1, Windows 7; Reformatting may be required for other operating systems; Compatibility may vary depending on user’s hardware configuration and operating system', 'title': 'WD 2TB Elements Portable External Hard Drive - USB 3.0 ', 'rating_count': 203, 'category': 'electronics', 'price': 64.0, 'rating_rate': 3.3}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000167B1324050>, 'id': 10, 'description': 'Easy upgrade for faster boot up, shutdown, application load and response (As compared to 5400 RPM SATA 2.5” hard drive; Based on published specifications and internal benchmarking tests using PCMark vantage scores) Boosts burst write performance, making it ideal for typical PC workloads The perfect balance of

In [ ]:
filtered_products = session.query(Product).filter(
    Product.category == 'electronics', Product.price > 50
).order_by(Product.rating_rate.desc()).all()

for product in filtered_products:
    print(product.__dict__)


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000167B131AB90>, 'id': 11, 'description': '3D NAND flash are applied to deliver high transfer speeds Remarkable transfer speeds that enable faster bootup and improved overall system performance. The advanced SLC Cache Technology allows performance boost and longer lifespan 7mm slim design suitable for Ultrabooks and Ultra-slim notebooks. Supports TRIM command, Garbage Collection technology, RAID, and ECC (Error Checking & Correction) to provide the optimized performance and enhanced reliability.', 'title': 'Silicon Power 256GB SSD 3D NAND A55 SLC Cache Performance Boost SATA III 2.5', 'rating_count': 319, 'category': 'electronics', 'price': 109.0, 'rating_rate': 4.8}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000167B131A310>, 'id': 12, 'description': "Expand your PS4 gaming experience, Play anywhere Fast and easy, setup Sleek design with high capacity, 3-year manufacturer's limited wa

Average price per product.

In [ ]:
from sqlalchemy import func

avg_price = session.query(func.avg(Product.price)).scalar()

print(f"Average price: {avg_price}")


Average price: 162.04599999999994
